PART 1: CUSTOM vs SCIKIT-LEARN DECISION TREE (CLASSIFICATION)

Step 1: Import Required Libraries

In [18]:
import numpy as np
import pandas as pd

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score


Step 2: Custom Decision Tree Implementation (Information Gain)

In [19]:
class CustomDecisionTree:
    def __init__(self, max_depth=None):
        self.max_depth = max_depth
        self.tree = None

    def fit(self, X, y):
        self.tree = self._build_tree(X, y)

    def _build_tree(self, X, y, depth=0):
        num_samples, num_features = X.shape
        unique_classes = np.unique(y)

        # Stopping conditions
        if len(unique_classes) == 1:
            return {'class': unique_classes[0]}

        if self.max_depth is not None and depth >= self.max_depth:
            return {'class': np.bincount(y).argmax()}

        best_gain = -1
        best_split = None

        for feature_idx in range(num_features):
            thresholds = np.unique(X[:, feature_idx])
            for threshold in thresholds:
                left_mask = X[:, feature_idx] <= threshold
                right_mask = ~left_mask

                if len(y[left_mask]) == 0 or len(y[right_mask]) == 0:
                    continue

                gain = self._information_gain(y, y[left_mask], y[right_mask])

                if gain > best_gain:
                    best_gain = gain
                    best_split = {
                        'feature_idx': feature_idx,
                        'threshold': threshold,
                        'left_mask': left_mask,
                        'right_mask': right_mask
                    }

        if best_split is None:
            return {'class': np.bincount(y).argmax()}

        left_tree = self._build_tree(
            X[best_split['left_mask']], y[best_split['left_mask']], depth + 1
        )
        right_tree = self._build_tree(
            X[best_split['right_mask']], y[best_split['right_mask']], depth + 1
        )

        return {
            'feature_idx': best_split['feature_idx'],
            'threshold': best_split['threshold'],
            'left_tree': left_tree,
            'right_tree': right_tree
        }

    def _information_gain(self, parent, left, right):
        parent_entropy = self._entropy(parent)
        left_entropy = self._entropy(left)
        right_entropy = self._entropy(right)

        weighted_entropy = (
            (len(left) / len(parent)) * left_entropy +
            (len(right) / len(parent)) * right_entropy
        )

        return parent_entropy - weighted_entropy

    def _entropy(self, y):
        probabilities = np.bincount(y) / len(y)
        return -np.sum(probabilities * np.log2(probabilities + 1e-9))

    def predict(self, X):
        return [self._predict_single(x, self.tree) for x in X]

    def _predict_single(self, x, tree):
        if 'class' in tree:
            return tree['class']

        if x[tree['feature_idx']] <= tree['threshold']:
            return self._predict_single(x, tree['left_tree'])
        else:
            return self._predict_single(x, tree['right_tree'])


Step 3: Load and Split the IRIS Dataset

In [20]:
# Load dataset
data = load_iris()
X = data.data
y = data.target

# Train-Test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


Step 4: Train & Evaluate Custom Decision Tree

In [21]:
custom_tree = CustomDecisionTree(max_depth=3)
custom_tree.fit(X_train, y_train)

y_pred_custom = custom_tree.predict(X_test)
accuracy_custom = accuracy_score(y_test, y_pred_custom)

print(f"Custom Decision Tree Accuracy: {accuracy_custom:.4f}")


Custom Decision Tree Accuracy: 1.0000


Step 5: Train & Evaluate Scikit-Learn Decision Tree

In [22]:
sklearn_tree = DecisionTreeClassifier(max_depth=3, random_state=42)
sklearn_tree.fit(X_train, y_train)

y_pred_sklearn = sklearn_tree.predict(X_test)
accuracy_sklearn = accuracy_score(y_test, y_pred_sklearn)

print(f"Scikit-learn Decision Tree Accuracy: {accuracy_sklearn:.4f}")


Scikit-learn Decision Tree Accuracy: 1.0000


Step 6: Accuracy Comparison

In [23]:
print("Accuracy Comparison")
print(f"Custom Decision Tree: {accuracy_custom:.4f}")
print(f"Scikit-learn Decision Tree: {accuracy_sklearn:.4f}")


Accuracy Comparison
Custom Decision Tree: 1.0000
Scikit-learn Decision Tree: 1.0000


PART 2: ENSEMBLE METHODS & HYPERPARAMETER TUNING (CLASSIFICATION)

Step 1: Load Wine Dataset

In [24]:
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split

# Load Wine dataset
wine = load_wine()
X = wine.data
y = wine.target

# Stratified Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)


Step 2: Train Decision Tree & Random Forest (Classification)

In [25]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

# Decision Tree Classifier
dt_clf = DecisionTreeClassifier(random_state=42)
dt_clf.fit(X_train, y_train)

# Random Forest Classifier
rf_clf = RandomForestClassifier(random_state=42)
rf_clf.fit(X_train, y_train)

from sklearn.metrics import f1_score

# Predictions
dt_pred = dt_clf.predict(X_test)
rf_pred = rf_clf.predict(X_test)

# F1 Scores
dt_f1 = f1_score(y_test, dt_pred, average='weighted')
rf_f1 = f1_score(y_test, rf_pred, average='weighted')

print(f"Decision Tree F1 Score : {dt_f1:.4f}")
print(f"Random Forest F1 Score : {rf_f1:.4f}")


Decision Tree F1 Score : 0.9450
Random Forest F1 Score : 1.0000


Step 3: Hyperparameter Tuning using GridSearchCV (Random Forest)

In [26]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5, 10]
}

grid_search = GridSearchCV(
    estimator=RandomForestClassifier(random_state=42),
    param_grid=param_grid,
    scoring='f1_weighted',
    cv=5,          # StratifiedKFold used internally for classification
    n_jobs=-1
)

grid_search.fit(X_train, y_train)

print("Best Parameters (Classification):", grid_search.best_params_)
print("Best F1 Score:", grid_search.best_score_)


Best Parameters (Classification): {'max_depth': None, 'min_samples_split': 2, 'n_estimators': 50}
Best F1 Score: 0.985974025974026


PART 3: REGRESSION – DECISION TREE & RANDOM FOREST

Step 1: Load Dataset for Regression

In [27]:
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split

# Load Wine dataset
wine = load_wine()
X = wine.data
y = wine.target.astype(float)   # convert categorical target to continuous

# Train-test split (NO stratification for regression)
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42
)


Step 2:Train Regression Models

In [28]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

# Decision Tree Regressor
dt_reg = DecisionTreeRegressor(random_state=42)
dt_reg.fit(X_train, y_train)

# Random Forest Regressor
rf_reg = RandomForestRegressor(random_state=42)
rf_reg.fit(X_train, y_train)


RandomForestRegressor(random_state=42)

Step 3: Predictions and Performance Evaluation

Metric Used: Mean Squared Error (MSE)

In [29]:
from sklearn.metrics import mean_squared_error

# Predictions
dt_pred = dt_reg.predict(X_test)
rf_pred = rf_reg.predict(X_test)

# Mean Squared Error
dt_mse = mean_squared_error(y_test, dt_pred)
rf_mse = mean_squared_error(y_test, rf_pred)

print("Decision Tree Regressor MSE :", dt_mse)
print("Random Forest Regressor MSE :", rf_mse)


Decision Tree Regressor MSE : 0.16666666666666666
Random Forest Regressor MSE : 0.06483333333333333


Step 4: Identify Hyperparameters for Random Forest Regression

Selected three hyperparameters:

1. n_estimators – number of trees

2. max_depth – depth of each tree

3. min_samples_split – minimum samples to split a node

Step 5: Hyperparameter Tuning using RandomizedSearchCV

In [30]:
from sklearn.model_selection import RandomizedSearchCV

param_dist = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10]
}

random_search = RandomizedSearchCV(
    estimator=RandomForestRegressor(random_state=42),
    param_distributions=param_dist,
    n_iter=10,
    cv=5,
    scoring='neg_mean_squared_error',
    random_state=42,
    n_jobs=-1
)

random_search.fit(X_train, y_train)

print("Best Parameters (Regression):", random_search.best_params_)
print("Best MSE:", -random_search.best_score_)


Best Parameters (Regression): {'n_estimators': 300, 'min_samples_split': 2, 'max_depth': 10}
Best MSE: 0.04472559113300492


Key Points :

* Decision Trees split data using feature-based thresholds and can be implemented both from scratch and using Scikit-learn.

* Scikit-learn’s Decision Tree performs better due to optimized algorithms and pruning.

* Random Forest is an ensemble method that combines multiple decision trees to improve accuracy and reduce overfitting.

* F1 Score is an effective metric for multi-class classification as it balances precision and recall.

* GridSearchCV performs exhaustive hyperparameter tuning using cross-validation for classification models.

* RandomizedSearchCV is computationally efficient for tuning regression models with large datasets.

* In regression, Random Forest Regressor achieves lower MSE than Decision Tree Regressor due to ensemble averaging.

* Large MSE values occur because the target variable is large and errors are squared.

* Feature scaling is not required for tree-based models since they use threshold comparisons, not distance metrics.

* Hyperparameter tuning improves model generalization and performance stability.